In [9]:
!pip install fasttext # necesario para importar los modulos
!git clone https://github.com/elsonidoq/ml-practico-2022.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'ml-practico-2022' already exists and is not an empty directory.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import sys
from google.colab import drive

sys.path.append('ml-practico-2022/lib')
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
from taller_model_selection.evaluate import load_train_dev_test

(X_train, y_train), (X_dev, y_dev), test = load_train_dev_test('/content/gdrive/MyDrive/taller-model-selection-data')

{'pct(train)': 0.7837289649483001, 'pct(dev)': 0.11952685477518159, 'pct(test)': 0.09674418027651828}


In [3]:
from taller_model_selection.transformers import FeatureProjection, PretrainedFastTextTransformer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


features_pipe = make_union(
    PretrainedFastTextTransformer('/content/gdrive/MyDrive/taller-model-selection-data/title_fasttext.bin', 'title'),
    PretrainedFastTextTransformer('/content/gdrive/MyDrive/taller-model-selection-data/desc_fasttext.bin', 'description'),
    make_pipeline(
        FeatureProjection(['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered']),
        SimpleImputer()
    ),
    make_pipeline(
        FeatureProjection(['l3']), 
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
    make_pipeline(
        FeatureProjection(['l4']), 
        SimpleImputer(strategy='constant'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
)

lr_pipe = make_pipeline(
    features_pipe,
    LinearRegression()
)


In [4]:
import lightgbm as lgb

lgb_pipe = make_pipeline(
    features_pipe,
    lgb.LGBMRegressor(n_estimators=100)
)

Si estas usando el encoder de la descripccion, el codigo de abajo toma 5 minutos en correr approx

In [5]:
lr_pipe.fit(X_train, y_train)
lgb_pipe.fit(X_train, y_train);

# Resultados agregando el titulo

In [14]:
from taller_model_selection.evaluate import Evaluator

ev = Evaluator(X_train, y_train, X_dev, y_dev)

# Hay que asegurarse que el pipeline de arriba tenga el titulo y no la descripccion
ev.eval_pipe('lr', lr_pipe)
ev.eval_pipe('rf', lgb_pipe)

pd.DataFrame(ev.evaluations)

,name,train,dev
0,lr,392472.789838,358645.591731
1,rf,239389.032664,267253.768555


# Resultados con titulo + descripccion

In [6]:
from taller_model_selection.evaluate import Evaluator

ev = Evaluator(X_train, y_train, X_dev, y_dev)

# Hay que asegurarse que el pipeline de arriba tenga el titulo y la descripccion
# La evaluacion toma al rededor de 5 minutos también
ev.eval_pipe('lr', lr_pipe)
ev.eval_pipe('rf', lgb_pipe)

pd.DataFrame(ev.evaluations)

,name,train,dev
0,lr,380659.911304,346740.224595
1,rf,217681.059686,249881.961576


In [ ]:
df = pd.DataFrame([eval_pipe('lr', lr_pipe), eval_pipe('rf', rf_pipe)])
df